<a href="https://colab.research.google.com/github/zSoftwareRepository/MDS556/blob/main/Random_Forrest_Classifier_Boruta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install boruta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 3.1 MB/s 


In [3]:
# importing libraries

import warnings
#warnings.filterwarnings("ignore")

seed = 23

import numpy as np
np.random.seed(seed)

import pandas as pd

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans

from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
features = ['ratio_open_30','prob_mix_ro_30t60','ratio_open_30t60','engaged_prob','engaged_index','avg_key_del_open','ratio_open_60','var_deliveries_last_60',
            'std_deliveries_last_90','prob_mix_ro_60t90','ratio_open_60t90','var_key_del_open','avg_deliveries_last_60','ratio_deliveries_last_60',
            'ratio_keydeliveries_last_60','ratio_deliveries_last_90','avg_deliveries_last_90','ratio_keydeliveries_last_90','avg_deliveries_last_180',
            'ratio_deliveries_last_180','std_key_del_close','ratio_keydeliveries_last_180','datediff_create_trans','datediff_month','ratio_open_90',
            'total_opens_last_90t180','app_ratio_open_90t180','app_total_opens_last_180','ratio_open_90t180','total_opens_last_180','app_total_opens_by_device',
            'age_1stperson_hushld','app_total_opens_last_90','avg_opens_by_device','std_opens_by_device','app_ratio_open_60t90','app_ratio_open_90',
            'app_ratio_open_60','app_total_opens_last_60','app_avg_opens_last_90','app_avg_opens_last_60','app_std_opens_last_60','app_ratio_open_30t60',
            'app_std_opens_last_30','app_avg_opens_last_30','app_ratio_open_30','app_total_opens_last_30','no_of_adults_hushld','home_lot_square_footage',
            'home_square_footage','vehicle_year_1st','home_loan_amount','gender']

#'auto_loans_index','bankcard_utilizn_index','zipcodecity'

In [5]:
def get_dummies(data,field,prefix):
    data[field] = data[field].fillna('na')
    dms = pd.get_dummies(data.loc[:,field], prefix=prefix)
    return pd.concat([data,dms],axis=1)

In [6]:
def toLog10(df,feature):    
    df[feature].fillna(0, inplace=True)
    return np.log10(df[feature]+1)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
data = pd.read_csv('drive/MyDrive/Elmhurst_Data/mydata.csv',low_memory=False)

In [9]:
target = data[['target']]
target.columns=['target']

In [10]:
data = data.replace(np.nan, 0)
data = data.replace('unknown', 0)
data.home_loan_amount = data.home_loan_amount * -1

In [11]:
#Create dummies for the gender feature
data = get_dummies(data,'gender','gnd')

In [12]:
#Binnarize and create dummies for the vehicle year feature
data['vehicle_year_bin'] = data['vehicle_year_1st'].apply(lambda x : 'new_model' if x >= 2019 else ('med_model' if (x < 2019 and x >= 2015) else 'old_model'))
data = get_dummies(data,'vehicle_year_bin','vyc')

In [14]:
drop_features = ['gender','vehicle_year_1st','vehicle_year_bin','target'] #,'auto_loans_index','bankcard_utilizn_index','zipcodecity']
data.drop(drop_features,inplace=True,axis=1)
print(data.shape)

(249575, 104)


In [15]:
# Split data 80% for training 20% for test

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=seed)
saved_cols = X_train.columns
y_train = y_train.values.ravel()

In [16]:
# Execute rendom forrest classifier

rf = RandomForestClassifier(n_estimators = 500, random_state=seed)

result = rf.fit(X_train, y_train)

In [17]:
# let's initialize Boruta
feat_selector = BorutaPy(
    verbose=2,
    estimator=rf,
    n_estimators='auto',
    max_iter=20  # number of iterations to perform
)

In [18]:
# train Boruta
# N.B.: X and y must be numpy arrays
feat_selector.fit(np.array(X_train), np.array(y_train))

# print support and ranking for each feature
print("\n------Support and Ranking for each feature------")
for i in range(len(feat_selector.support_)):
    if feat_selector.support_[i]:
        print("Passes the test: ", X_train.columns[i],
              " - Ranking: ", feat_selector.ranking_[i])
    else:
        print("Doesn't pass the test: ",
              X_train.columns[i], " - Ranking: ", feat_selector.ranking_[i])

Iteration: 	1 / 20
Confirmed: 	0
Tentative: 	104
Rejected: 	0
Iteration: 	2 / 20
Confirmed: 	0
Tentative: 	104
Rejected: 	0
Iteration: 	3 / 20
Confirmed: 	0
Tentative: 	104
Rejected: 	0
Iteration: 	4 / 20
Confirmed: 	0
Tentative: 	104
Rejected: 	0
Iteration: 	5 / 20
Confirmed: 	0
Tentative: 	104
Rejected: 	0
Iteration: 	6 / 20
Confirmed: 	0
Tentative: 	104
Rejected: 	0
Iteration: 	7 / 20
Confirmed: 	0
Tentative: 	104
Rejected: 	0
Iteration: 	8 / 20
Confirmed: 	14
Tentative: 	11
Rejected: 	79
Iteration: 	9 / 20
Confirmed: 	14
Tentative: 	11
Rejected: 	79
Iteration: 	10 / 20
Confirmed: 	14
Tentative: 	11
Rejected: 	79
Iteration: 	11 / 20
Confirmed: 	14
Tentative: 	11
Rejected: 	79
Iteration: 	12 / 20
Confirmed: 	14
Tentative: 	8
Rejected: 	82
Iteration: 	13 / 20
Confirmed: 	14
Tentative: 	8
Rejected: 	82
Iteration: 	14 / 20
Confirmed: 	14
Tentative: 	8
Rejected: 	82
Iteration: 	15 / 20
Confirmed: 	14
Tentative: 	8
Rejected: 	82
Iteration: 	16 / 20
Confirmed: 	14
Tentative: 	7
Rejected: 	